# Протоколы обмена ключами

In [1]:
IS_DEBUG = 1

In [2]:
def trace(*args, **kwargs):
    """
    Отладочная трассировка
    """
    
    global IS_DEBUG
    if IS_DEBUG:
        print('[TRACE]', end=' ')
        print(*args, **kwargs)

In [3]:
def bytes_as_hex(b: bytes) -> str:
    """
    Перевод бинарных данных в hex-строчку
    """
    
    return binascii.hexlify(b).decode()

In [4]:
def randint(a, b):
    """
    Генерация слуайного числа среди заданных.
    """
    
    native_a = int(a)
    native_b = int(b)
    return gmpy2.mpz(random.randint(native_a, native_b))

----

In [5]:
# Генерация простых чисел и КГПСЧ
from Crypto.Util.number import getStrongPrime
from Crypto.Random import random

# HKDF и то, что надо для нее
from Crypto.Protocol.KDF import HKDF
from Crypto.Hash import SHA512

# Подпись и шифрование
from Crypto.PublicKey import RSA
from Crypto.Signature import pss
from Crypto.Cipher import AES

# Математическая библиотека
import gmpy2

# Для генерации идентификаторов
import uuid

# Утилитки
from Crypto.Util.number import long_to_bytes
import binascii

----

## Протокол Диффи-Хеллмана
![Diffe-Hellman Protocol](./images/Diffie-Hellman_4.png)

In [6]:
class DiffieHellmanParams(object):
    def __init__(self, prime_bits: int = 512):
        """
        Генерация параметров протокола - простого числа и генератора.
        """
        
        self._p = gmpy2.mpz(getStrongPrime(prime_bits))
        self._g = randint(2, self._p - 2)
        
        trace('[DiffieHellmanParams]', f'p = {self._p}, g = {self._g}')
        
        
    @property
    def prime(self) -> int:
        """
        Получение простого числе - модуля.
        """
        
        return self._p
    
    
    @property
    def generator(self) -> int:
        """
        Получение генератора.
        """
        
        return self._g

In [7]:
class DiffieHellmanUser(object):
    KEY_LENGTH = 32
    HKDF_DUMMY_STUFF = b'\x00' * 16
    
    def __init__(self, params: DiffieHellmanParams, 
                 use_another_value: bool = False):
        """
        Инициализация клиента: генерация идентификатора,
        сохранение параметров протокола.
        Последний параметр служит для демонстрации неуспешного выполнения.
        """
        
        self._p = params.prime
        self._g = params.generator
        self._id = uuid.uuid4()
        self._key = None
        self._sabotage = use_another_value
        
        trace('[DiffieHellmanUser]', f'client {self._id} created')
    
    
    def initiate_key_exchange(self, another_client):
        """
        Инициирование протокола обмена ключами.
        """
        
        #
        # Генерирую х, отправляю другому пользователю
        # Получаю от него g ** y mod p
        #
        
        x = self._generate_group_element()
        trace('[DiffieHellmanUser]', f'[{self._id}]', f'{x = }')
        
        intermediate_value = another_client._accept_key_exchange(gmpy2.powmod(self._g, x, self._p))
        
        #
        # Если требуют неудачную попытку, то подменю x
        # Это как бы незнание изначального значения - попытка угадать
        #
        
        if self._sabotage:
            x = self._generate_group_element()
            trace('[DiffieHellmanUser]', f'[{self._id}]', f'guessed {x = }')
        
        #
        # Вывожу ключ
        #
        
        self._key = HKDF(DiffieHellmanUser.HKDF_DUMMY_STUFF, 
                         DiffieHellmanUser.KEY_LENGTH, 
                         long_to_bytes(int(gmpy2.powmod(intermediate_value, x, self._p))), 
                         SHA512, 
                         1)
        
        trace('[DiffieHellmanUser]', f'[{self._id}]', f'derived key: {bytes_as_hex(self._key)}')
        
        
    def _accept_key_exchange(self, intermediate_value) -> int:
        #
        # Получаю g ** x mod p, генерирую у
        #
        
        y = self._generate_group_element()
        trace('[DiffieHellmanUser]', f'[{self._id}]', f'{y = }')
        
        #
        # Вывожу ключ
        #
        
        self._key = HKDF(DiffieHellmanUser.HKDF_DUMMY_STUFF, 
                         DiffieHellmanUser.KEY_LENGTH, 
                         long_to_bytes(int(gmpy2.powmod(intermediate_value, y, self._p))), 
                         SHA512, 
                         1)
        
        trace('[DiffieHellmanUser]', f'[{self._id}]', f'derived key: {bytes_as_hex(self._key)}')
        
        return gmpy2.powmod(self._g, y, self._p)
    
    
    def _generate_group_element(self) -> int:
        return randint(1, self._p - 2)

----

In [8]:
# Параметры протокола
params = DiffieHellmanParams()

[TRACE] [DiffieHellmanParams] p = 12282544736328760243026765634890456745649694458194767574123232665959514403419346116797704289188414201604635672014063008789798549388335825098332193566704797, g = 10925865511008462065940475859916686780147048993273375704219582392588567981685187999227844529584833055127825668524267784779163578784564251759684142902403117


In [9]:
# Два хороших пользователя
alice = DiffieHellmanUser(params)
bob   = DiffieHellmanUser(params)

[TRACE] [DiffieHellmanUser] client d1e30be5-916b-487f-9f38-6c977ed0bb74 created
[TRACE] [DiffieHellmanUser] client c967f198-6e8a-4ecc-a867-fef8b742cb1b created


In [10]:
# Ключ получается установить успешно
alice.initiate_key_exchange(bob)

[TRACE] [DiffieHellmanUser] [d1e30be5-916b-487f-9f38-6c977ed0bb74] x = mpz(10431010886124306605883737547944643884183695693442653442814903165591466306263465074339781219933276130768722896866386445925327699341516938983563751157130292)
[TRACE] [DiffieHellmanUser] [c967f198-6e8a-4ecc-a867-fef8b742cb1b] y = mpz(10042283536881144234419443339394361633205346966070313499004462852462941891651813007421152046874502453828638696293025083542361423137876594358271949170863294)
[TRACE] [DiffieHellmanUser] [c967f198-6e8a-4ecc-a867-fef8b742cb1b] derived key: 19abf0451f86a2bd4ec75e02a16a7769013acb5fdc450dc4cf564888ebbaa6fc
[TRACE] [DiffieHellmanUser] [d1e30be5-916b-487f-9f38-6c977ed0bb74] derived key: 19abf0451f86a2bd4ec75e02a16a7769013acb5fdc450dc4cf564888ebbaa6fc


In [11]:
# Теперь врага создам
mallory = DiffieHellmanUser(params, True)

[TRACE] [DiffieHellmanUser] client 326c9750-7ffb-428f-b3c1-ea4545021245 created


In [12]:
# Внедрившись в исполняющийся протокол, врагу не достается правильный ключ
mallory.initiate_key_exchange(bob)

[TRACE] [DiffieHellmanUser] [326c9750-7ffb-428f-b3c1-ea4545021245] x = mpz(11789907413264189037850934253699398970062704660372196165510963954823720508960930923338089022758816733841882983634756784943788901182112094324853548379070760)
[TRACE] [DiffieHellmanUser] [c967f198-6e8a-4ecc-a867-fef8b742cb1b] y = mpz(7037421268436614293814866482151849758016757522815345211472780571712879458780182438157238438453619404359842442295151950832620387843063807076533862350428456)
[TRACE] [DiffieHellmanUser] [c967f198-6e8a-4ecc-a867-fef8b742cb1b] derived key: 7c970b28da6200e7b8097a89a2718088495a50e9cf89454ada3305fc59859c13
[TRACE] [DiffieHellmanUser] [326c9750-7ffb-428f-b3c1-ea4545021245] guessed x = mpz(7370267559285802039535242595444042620139183388346514608998860728975560500309202457033576044730929726136595068012277563392968617969383313336927550890222661)
[TRACE] [DiffieHellmanUser] [326c9750-7ffb-428f-b3c1-ea4545021245] derived key: 3409fd42564707e77fc0b560cfe9c93786138bef51eb76d15715a120e7cf6936


----
## Протокол MTI
![Diffe-Hellman Protocol](./images/MTI_4.png)

In [13]:
class MTIParams(object):
    def __init__(self, prime_bits):
        """
        Генерация параметров протокола - простого числа и генератора.
        """
        
        self._p = gmpy2.mpz(getStrongPrime(prime_bits))
        self._g = randint(2, self._p - 2)
        
        trace('[MTIParams]', f'p = {self._p}, g = {self._g}')
        
        
    @property
    def prime(self) -> int:
        """
        Получение простого числе - модуля.
        """
        
        return self._p
    
    
    @property
    def generator(self) -> int:
        """
        Получение генератора.
        """
        
        return self._g

In [14]:
class MTIUser(object):
    KEY_LENGTH = 32
    HKDF_DUMMY_STUFF = b'\x00' * 16
    
    def __init__(self, params: MTIParams, *,
                 fake_data: gmpy2.mpz = None):
        """
        Инициализация клиента: генерация идентификатора,
        сохранение параметров протокола, генерация ключевой пары.
        Последний параметр служит для демонстрации неуспешного выполнения.
        """
        
        self._p = params.prime
        self._g = params.generator
        self._id = uuid.uuid4()
        self._key = None
        
        self._private_key = self._generate_group_element()
        self._public_key  = gmpy2.powmod(self._g, self._private_key, self._p) if fake_data is None else fake_data
        
        trace('[MTIUser]', f'client {self._id} created. PK = {self._public_key}, SK = {self._private_key}')
        
        
    def initiate_key_exchange(self, another_client):
        """
        Инициирование протокола обмена ключами.
        """
        
        #
        # Генерирую х, вычисляю g ** x mod p
        #
        
        x = self._generate_group_element()
        m = gmpy2.powmod(self._g, x, self._p)
        
        trace('[MTIUser]', f'[{self._id}]', f'{x = }, {m = }')
        
        #
        # Отправляю другому клиенту и получаю от него g ** y mod p
        #
        
        intermediate_value = another_client._accept_key_exchange(m, self.public_key)
        
        #
        # Вывожу ключ
        #
        
        product = gmpy2.powmod(another_client.public_key, x, self._p) * \
            gmpy2.powmod(intermediate_value, self._private_key, self._p)
        
        self._key = HKDF(DiffieHellmanUser.HKDF_DUMMY_STUFF, 
                         DiffieHellmanUser.KEY_LENGTH, 
                         long_to_bytes(int(gmpy2.t_mod(product, self._p))), 
                         SHA512, 
                         1)
        
        trace('[MTIUser]', f'[{self._id}]', f'derived key: {bytes_as_hex(self._key)}')
    
    
    def _accept_key_exchange(self, intermediate_value, public_key):
        #
        # Генерирую у, вычисляю g ** y mod p
        #
        
        y = self._generate_group_element()
        m = gmpy2.powmod(self._g, y, self._p)
        
        trace('[MTIUser]', f'[{self._id}]', f'{y = }, {m = }')
        
        #
        # Вывожу ключ
        #
        
        product = gmpy2.powmod(public_key, y, self._p) * \
            gmpy2.powmod(intermediate_value, self._private_key, self._p)
        
        self._key = HKDF(MTIUser.HKDF_DUMMY_STUFF, 
                         MTIUser.KEY_LENGTH, 
                         long_to_bytes(int(gmpy2.t_mod(product, self._p))), 
                         SHA512, 
                         1)
        
        trace('[MTIUser]', f'[{self._id}]', f'derived key: {bytes_as_hex(self._key)}')
        
        return m
        
    
    @property
    def public_key(self):
        """
        Получение открытого ключа.
        """
        
        return self._public_key
    
    
    def _generate_group_element(self) -> int:
        return randint(1, self._p - 2)

----

In [15]:
# Параметры протокола
params = MTIParams(512)

[TRACE] [MTIParams] p = 9725381632972656754029716451582880443463444738124527858271030760827245237712655757679088062820499853144544361077243067748399916401798581906755757652363081, g = 9219202535250546584204254299901899076751160645031454977540429440278324095127001361985710406330153588518017271306143162930947125787065942017998706511561781


In [16]:
# Два хороших пользователя
alice = MTIUser(params)
bob   = MTIUser(params)

[TRACE] [MTIUser] client cffe6362-90be-436a-813a-00e0bb82164d created. PK = 2952853493538183630828625043496948730764883723180629425061231778369297207436467672514885057938128701489128236525397272132687497012189246901640953245990961, SK = 2700938011662816570262962387618941290818334717066721608534830291435999760048521084883810520793699831530440991965336763589879991140529802064499774547244641
[TRACE] [MTIUser] client 029e36d7-f6d3-4f36-88ba-cbb38cae962c created. PK = 346315248966047974271593497887760195455512774034293663796578561369523200504864909900259678583482221771488779813577156815113098165883621554540346657597287, SK = 1839382171983581202101232555151176465479842288283921463427668509160048632173639599083076328884270636344713974431719964847171753049230563528705344396367112


In [17]:
# Ключ получается установить успешно
alice.initiate_key_exchange(bob)

[TRACE] [MTIUser] [cffe6362-90be-436a-813a-00e0bb82164d] x = mpz(3786960327804069178893009713212459981524278468688772522040815731805454319172908889237396895330979760490954151690104612298522090065347528172308168036708301), m = mpz(5973500247205414625375323888273118756858424268085836043950084092645916111385374463986586582357172979341166074937566060190560348018780045485402395900020642)
[TRACE] [MTIUser] [029e36d7-f6d3-4f36-88ba-cbb38cae962c] y = mpz(8364474421153747972971977326119283753295466886522626446420136480457794182719935699182793898310080974811380369273990601449364240261073085915954850990534358), m = mpz(5229681888743075214882230678838011214975289901358445740600288083014850853304339591520590598585915958173478109500389594599233589811594435611189458932110132)
[TRACE] [MTIUser] [029e36d7-f6d3-4f36-88ba-cbb38cae962c] derived key: 1ce1587eed270015a7550efc6fbb0f4303d0b868f854935c4ab4d95a3b9c1adc
[TRACE] [MTIUser] [cffe6362-90be-436a-813a-00e0bb82164d] derived key: 1ce1587eed270015a7550ef

In [18]:
# Теперь врага создам
mallory = MTIUser(params, fake_data=alice.public_key)

[TRACE] [MTIUser] client a1e3edd6-10bd-48ea-9259-843a594cf0f5 created. PK = 2952853493538183630828625043496948730764883723180629425061231778369297207436467672514885057938128701489128236525397272132687497012189246901640953245990961, SK = 690137509559984730751325724246643733924862322211435911847480866723253482528867744067712967444987198139359696996484997875276256103627865879320261084472578


In [19]:
# Внедрившись в исполняющийся протокол, врагу не достается правильный ключ
mallory.initiate_key_exchange(bob)

[TRACE] [MTIUser] [a1e3edd6-10bd-48ea-9259-843a594cf0f5] x = mpz(7995555341800226394576150980043716826242724140017965200851821592559917434643473037089543420045178010537663414704320934259338035966168981458514511776436742), m = mpz(7555209577107157203372767627848061249057766587468511830268218433394432650296368364962835939770468632364788248243625135451367690918942844464077885298279536)
[TRACE] [MTIUser] [029e36d7-f6d3-4f36-88ba-cbb38cae962c] y = mpz(1217262847213467250755990909993593173735694562615815988529850982667463584851894240705823559109957325480997405949020512831325026617602168973081108075253571), m = mpz(6919117170187863751070297754589981542800143173971315103867320361352474376497464566510072338255306571495759014668940154638402422495413922026959402627185671)
[TRACE] [MTIUser] [029e36d7-f6d3-4f36-88ba-cbb38cae962c] derived key: 63a36f09bdaeb7390fa353d65fad973898a7edbbf19e4282baad5d428a6784af
[TRACE] [MTIUser] [a1e3edd6-10bd-48ea-9259-843a594cf0f5] derived key: 9c97387f3cd41175414b2be

----
## Протокол STS
![STS Protocol](./images/STS_4.png)

In [20]:
class STSParams(object):
    def __init__(self, prime_bits):
        """
        Генерация параметров протокола - простого числа и генератора.
        """
        
        self._p = gmpy2.mpz(getStrongPrime(prime_bits))
        self._g = randint(2, self._p - 2)
        
        trace('[STSParams]', f'p = {self._p}, g = {self._g}')
        
        
    @property
    def prime(self) -> int:
        """
        Получение простого числе - модуля.
        """
        
        return self._p
    
    
    @property
    def generator(self) -> int:
        """
        Получение генератора.
        """
        
        return self._g

In [21]:
class STSUser(object):
    RSA_KEY_LENGTH = 4096
    KEY_LENGTH = 32
    HKDF_DUMMY_STUFF = b'\x00' * 16
    
    def __init__(self, params: STSParams, *, 
                 fake_data=None):
        """
        Инициализация клиента: генерация идентификатора,
        сохранение параметров протокола, генерация ключевой пары.
        Последний параметр служит для демонстрации неуспешного выполнения.
        """
        
        self._p = params.prime
        self._g = params.generator
        self._id = uuid.uuid4()
        self._key = None
        
        self._private_key = RSA.generate(STSUser.RSA_KEY_LENGTH)
        self._public_key  = self._private_key.publickey() if fake_data is None else fake_data
        
        trace('[STSUser]', f'client {self._id} created. PK = {self._public_key}, SK = {self._private_key}')
        
    
    def initiate_key_exchange(self, another_client):
        """
        Инициирование протокола обмена ключами.
        """
        
        #
        # Генерирую х, вычисляю g ** x mod p
        #
        
        x = self._generate_group_element()
        m = gmpy2.powmod(self._g, x, self._p)
        
        trace('[STSUser]', f'[{self._id}]', f'{x = }, {m = }')
        
        #
        # Отправляю другому пользователю, получаю g ** y mod p и зашифрованную подпись
        #
        
        intermediate_value, encrypted_signature = another_client._accept_key_exchange(m)
        
        #
        # Вывожу ключ
        #
        
        self._key = HKDF(DiffieHellmanUser.HKDF_DUMMY_STUFF, 
                         DiffieHellmanUser.KEY_LENGTH, 
                         long_to_bytes(int(gmpy2.powmod(intermediate_value, x, self._p))), 
                         SHA512, 
                         1)
        
        #
        # Расшифровываю подпись и проверяю ее
        # Если не подошла, то ключ неверный, завершаюсь
        #
        
        nonce, ct = encrypted_signature
        
        cipher = AES.new(self._key, AES.MODE_CTR, nonce=nonce)
        received_signature = cipher.decrypt(ct)
        
        if not self._check_signature(intermediate_value, m, received_signature, another_client.public_key):
            trace('[STSUser]', f'[{self._id}]', 'Invalid signature')
            self._key = None
            return
        
        #
        # Сам теперь подписываю значения и отправляю на проверку другому пользователю
        #
        
        signature = self._sign_values(m, intermediate_value)
        
        cipher = AES.new(self._key, AES.MODE_CTR)
        if not another_client._check_key_exchange(m, intermediate_value, 
                                                  (cipher.nonce, cipher.encrypt(signature)), 
                                                  self.public_key):
            trace('[STSUser]', f'[{self._id}]', 'Another user refused key exchange')
            self._key = None
            return
        
        #
        # Тут все хорошо
        #
        
        trace('[STSUser]', f'[{self._id}]', f'derived key: {bytes_as_hex(self._key)}')
        
        
    def _accept_key_exchange(self, intermediate_value):
        #
        # Генерирую у, вычисляю g ** y mod p
        # Вывожу ключ
        #
        
        y = self._generate_group_element()
        m = gmpy2.powmod(self._g, y, self._p)
        
        trace('[STSUser]', f'[{self._id}]', f'{y = }, {m = }')
        
        self._key = HKDF(DiffieHellmanUser.HKDF_DUMMY_STUFF, 
                         DiffieHellmanUser.KEY_LENGTH, 
                         long_to_bytes(int(gmpy2.powmod(intermediate_value, y, self._p))), 
                         SHA512, 
                         1)
        
        #
        # Создаю подпись значений и отправляю инициатору
        #
        
        signature = self._sign_values(m, intermediate_value)
        
        cipher = AES.new(self._key, AES.MODE_CTR)
        encrypted_signature = (cipher.nonce, cipher.encrypt(signature))
        
        return m, encrypted_signature
    
    
    def _check_key_exchange(self, f, s, encrypted_signature, pk):
        #
        # Проверяю подпись от инициатора
        #
        
        nonce, ct = encrypted_signature
        
        cipher = AES.new(self._key, AES.MODE_CTR, nonce=nonce)
        received_signature = cipher.decrypt(ct)
        
        if not self._check_signature(f, s, received_signature, pk):
            trace('[STSUser]', f'[{self._id}]', 'Invalid signature')
            self._key = None
            return False
        
        #
        # Если все валидно, то ключ верно установлен
        #
        
        trace('[STSUser]', f'[{self._id}]', f'derived key: {bytes_as_hex(self._key)}')
        return True
        
    
    def _sign_values(self, f, s):
        first  = long_to_bytes(int(f))
        second = long_to_bytes(int(s))
        
        h = SHA512.new(first + second)
        return pss.new(self._private_key).sign(h)
    
    
    def _check_signature(self, f, s, signature, pk):
        first  = long_to_bytes(int(f))
        second = long_to_bytes(int(s))
        
        h = SHA512.new(first + second)
        verifier = pss.new(pk)
        
        try:
            verifier.verify(h, signature)
            return True
        except:
            return False
    
        
    def _generate_group_element(self) -> int:
        return randint(1, self._p - 2)
        
    
    @property
    def public_key(self):
        """
        Получение открытого ключа.
        """
        
        return self._public_key

-----

In [22]:
# Параметры протокола
params = STSParams(512)

[TRACE] [STSParams] p = 11232274249738532142131489196935121225910440566106011552439303439926928013431795415023356298133756513469212160547911965091156686346242297070687893093153171, g = 3703724784094297755726611808167617048764141338548201271782495733528312787746892129615227558186123143477223122615486361846760948743567779645122050439885060


In [23]:
# Два хороших пользователя
alice = STSUser(params)
bob   = STSUser(params)

[TRACE] [STSUser] client d9240911-a58b-4ca5-a5d1-841269c00b2c created. PK = Public RSA key at 0x105F4E8E0, SK = Private RSA key at 0x105870130
[TRACE] [STSUser] client 002112bd-8267-465a-8300-4f231594c641 created. PK = Public RSA key at 0x105879760, SK = Private RSA key at 0x10584C4F0


In [24]:
# Ключ получается установить успешно
alice.initiate_key_exchange(bob)

[TRACE] [STSUser] [d9240911-a58b-4ca5-a5d1-841269c00b2c] x = mpz(7831660637058452491378644603432080251112946726494887144414896323732226564221238013955871441553273513915778876578858587888470860688550682324538861869514285), m = mpz(8901807572445272275848899344472250252722793646057584217626191951970227517598969604325173787167202364486859057893694010527045794681068982261652353514767220)
[TRACE] [STSUser] [002112bd-8267-465a-8300-4f231594c641] y = mpz(3230529102521213257897934573774279546847363030940912061508019076461639526749782984476699288731517889563341641220904662536146888465368638621903297850674794), m = mpz(5709691589810202398655810993548362162961301780321124485438441496190791521458212715364743685580809378487904578714328193313055486146296268725014153975099136)
[TRACE] [STSUser] [002112bd-8267-465a-8300-4f231594c641] derived key: 3b156ff731033a1dbb548427600e173214ecf3263fac974433adcad128e08b5a
[TRACE] [STSUser] [d9240911-a58b-4ca5-a5d1-841269c00b2c] derived key: 3b156ff731033a1dbb54842

In [25]:
# Теперь врага создам
mallory = STSUser(params, fake_data=alice.public_key)

[TRACE] [STSUser] client 4c578aef-a4aa-4e1b-b628-17d0ce8df77f created. PK = Public RSA key at 0x105F4E8E0, SK = Private RSA key at 0x105F4EF10


In [26]:
# Внедрившись в исполняющийся протокол, врагу не достается правильный ключ
mallory.initiate_key_exchange(bob)

[TRACE] [STSUser] [4c578aef-a4aa-4e1b-b628-17d0ce8df77f] x = mpz(755066909248152896126137729314431483791142047286248595324115149810626328133869708358546329406813274771296106798674982393450186507220355979212961845365093), m = mpz(4304284707874270719009952586617966460136494145518335133021196347820687607888207811938380894592089582340292587919137508625893847807607700557095276485860677)
[TRACE] [STSUser] [002112bd-8267-465a-8300-4f231594c641] y = mpz(5758418179373110055379904448101764138530162508641947619689202319930747524003942365061542432804102583486811983362997889122770283130968170472548908561956536), m = mpz(2304574303382415019499484910492670515699013308413732770504144703624959264777363401810577905519819977046340189927792409982896026626966299338530631997699281)
[TRACE] [STSUser] [002112bd-8267-465a-8300-4f231594c641] Invalid signature
[TRACE] [STSUser] [4c578aef-a4aa-4e1b-b628-17d0ce8df77f] Another user refused key exchange


In [27]:
bob.initiate_key_exchange(mallory)

[TRACE] [STSUser] [002112bd-8267-465a-8300-4f231594c641] x = mpz(5013385743649756632991046704695886714915773684913239304088030973370119421465220544014610146166735028730586432115888163011655963725709801450324375847296410), m = mpz(10003012801294067658955742932543537200060798074373356073729958666612767061805221072039710859382613263715101235770334797183340974966938305572312624069577387)
[TRACE] [STSUser] [4c578aef-a4aa-4e1b-b628-17d0ce8df77f] y = mpz(10762089541322928704393166619001793100546662178987982422082789374018582691323158119849235812885394435909849382853428591497642407714586887441617765727684752), m = mpz(4264451241335789824966067194167116395781380855510121771257097089229437765342442957732233051318929076539357226260775561339717145090595622291675835913808882)
[TRACE] [STSUser] [002112bd-8267-465a-8300-4f231594c641] Invalid signature
